In [1]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

In [3]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium["EX_fru_e"]= 0
medium["EX_co2_e"]= 1000
medium["EX_co_e"]= 0
medium["EX_h2_e"]= 10
Cljung_Model.medium = medium
#solution = Cljung_Model.optimize()

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


# Determine minimal set of reactions for acetogenesis:

In [4]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium["EX_fru_e"]= 0
medium["EX_co2_e"]= 1000
medium["EX_co_e"]= 10
medium["EX_h2_e"]= 0
Cljung_Model.medium = medium

Cljung_Model.reactions.ATPM.bounds = 0, 1000

Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0

Cljung_Model.reactions.ACt2r.knock_out()

acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    Cljung_Model.metabolites.ac_c: -1.0,
    Cljung_Model.metabolites.ac_e: 1.0
})
acT

#Cljung_Model.reactions.ACACT1r.knock_out()

Cljung_Model.add_reaction(acT)

#Cljung_Model.objective = "ATPS4r"
Cljung_Model.objective = "EX_ac_e"
#Cljung_Model.objective = "EX_etoh_e"
#Cljung_Model.objective = "EX_but_e"
#Cljung_Model.objective = "EX_btoh_e"
#Cljung_Model.objective = "EX_btd_RR_e"
print(Cljung_Model.slim_optimize())
solution = Cljung_Model.optimize()

solution.fluxes[solution.fluxes != 0].index

2.5


Index(['EX_h2o_e', 'EX_ac_e', 'EX_co_e', 'EX_co2_e', 'ACKr', 'ATPM', 'EX_h_e',
       'CO2t', 'COt', 'ALCD4y', 'CODH4', 'CODH_ACS', 'H2Ot', 'FDH7', 'FTHFLi',
       'METR', 'MTHFD', 'MTHFC', 'MTHFR5', 'PTAr', 'RNF', 'ALCD4', 'ATPS4r',
       'ACt'],
      dtype='object')

In [5]:
Cljung_Model.reactions.ATPS4r

Reaction identifier,ATPS4r
Name,ATP synthase (four protons for one ATP)
Memory address,0x07fceccec1240
Stoichiometry,adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c ADP C10H12N5O10P2 + 4.0 H+ + Phosphate <=> ATP C10H12N5O13P3 + H2O H2O + 3.0 H+
GPR,( CLJU_RS01160 and CLJU_RS01165 and CLJU_RS01170 ) and ( CLJU_RS01175 and CLJU_RS01180 and...
Lower bound,-1000.0
Upper bound,1000.0


In [6]:
Builder(model = Cljung_Model, map_json = join(data_dir,'Models/EscherMaps/WL_core_clean.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': 0.0, 'EX_h2o_e': -5.0, 'EX_his__L_e': 0.0, 'EX_ile__L_e': 0.0, 'EX_lac__D_e'…

In [7]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_h_e,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ACt,

Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ADK1,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.H2td,

Cljung_Model.reactions.ASPT,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,

Cljung_Model.reactions.FRNDPR2r,
Cljung_Model.reactions.POR,
Cljung_Model.reactions.PFL,
    
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FUM,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.GLUDy,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.HYDFDN2r,
Cljung_Model.reactions.MDH,
Cljung_Model.reactions.MTHFD,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.RNF,
Cljung_Model.reactions.PPA,
Cljung_Model.reactions.ASPTA,
Cljung_Model.reactions.ATPS4r,
Cljung_Model.reactions.NTPP6])

WLmodel.objective = "EX_ac_e"
print(WLmodel.slim_optimize())
solution = WLmodel.optimize()
Builder(model = WLmodel, map_json = join(data_dir,'Models/EscherMaps/WL_core.json'), reaction_data = solution.fluxes.to_dict())

2.5


Builder(reaction_data={'EX_h2_e': 0.0, 'EX_h2o_e': -5.0, 'EX_ac_e': 2.5, 'EX_co2_e': 5.0, 'EX_co_e': -10.0, 'E…

In [8]:
for x in Cljung_Model.reactions:
    if x.bounds == (0, 0):{
        print(x.id)
    }

ACt2r
HYDFDi
HYDFDN


with Cljung_Model:
    Cljung_Model.objective = "EX_ac_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    print("Acetate production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)
    
with Cljung_Model:
    Cljung_Model.objective = "EX_etoh_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    print("Ethanol production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)

with Cljung_Model:
    Cljung_Model.objective = "EX_but_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.BUTKr.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.PBUTT.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACOAD1z.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ECOAH1.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.HACD1.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACACT1r.bounds = -1000.0, 1000.0
    print("Butyrate production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)
    
with Cljung_Model:
    Cljung_Model.objective = "EX_btoh_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ALCD4.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.BTCOARx.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACOAD1z.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ECOAH1.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.HACD1.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACACT1r.bounds = -1000.0, 1000.0
    print("Butanol production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)
    
with Cljung_Model:
    Cljung_Model.objective = "EX_btd_RR_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    Cljung_Model.reactions.BTDD_RR.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.BTDt_RR.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACLDC.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACLS.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.POR.bounds = -1000.0, 1000.0
    print("Butanediol production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)

In [9]:
Builder(model = Cljung_Model, map_json = join(data_dir,'Models/EscherMaps/WL_core.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': 0.0, 'EX_h2o_e': -5.0, 'EX_ac_e': 2.5, 'EX_co2_e': 5.0, 'EX_co_e': -10.0, 'E…

In [10]:
Cljung_Model.reactions.EX_h2_e
Cljung_Model.reactions.EX_h2o_e
Cljung_Model.reactions.EX_ac_e
Cljung_Model.reactions.EX_co2_e
Cljung_Model.reactions.ACt2r
Cljung_Model.reactions.ACKr
Cljung_Model.reactions.ADK1
Cljung_Model.reactions.ATPM
Cljung_Model.reactions.EX_h_e
Cljung_Model.reactions.CO2t
Cljung_Model.reactions.CODH_ACS
Cljung_Model.reactions.H2Ot
Cljung_Model.reactions.H2td
Cljung_Model.reactions.FDH7
Cljung_Model.reactions.FRNDPR2r
Cljung_Model.reactions.FTHFLi
Cljung_Model.reactions.METR
Cljung_Model.reactions.HYDFDN2r
Cljung_Model.reactions.MTHFD
Cljung_Model.reactions.MTHFC
Cljung_Model.reactions.MTHFR5
Cljung_Model.reactions.PTAr
Cljung_Model.reactions.RNF
Cljung_Model.reactions.PPA
Cljung_Model.reactions.ATPS4r
Cljung_Model.reactions.NTPP6

Reaction identifier,NTPP6
Name,Nucleoside triphosphate pyrophosphorylase (atp)
Memory address,0x07fcecd13dc88
Stoichiometry,atp_c + h2o_c --> amp_c + h_c + ppi_c ATP C10H12N5O13P3 + H2O H2O --> AMP C10H12N5O7P + H+ + Diphosphate
GPR,CLJU_RS20535
Lower bound,0.0
Upper bound,1000.0


In [11]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport & Exchange:
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_pi_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_etoh_e,
Cljung_Model.reactions.EX_btd_RR_e,
Cljung_Model.reactions.EX_but_e,
Cljung_Model.reactions.EX_btoh_e,
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
#Cljung_Model.reactions.ACt2r,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,
#Energy:
Cljung_Model.reactions.HYDFDN,
Cljung_Model.reactions.HYDFDi,
Cljung_Model.reactions.HYDFDN2r,
Cljung_Model.reactions.ATPS4r,
Cljung_Model.reactions.ATPM,
Cljung_Model.reactions.RNF,#Add this reaction in Escher
Cljung_Model.reactions.NTPP6,#Add this reaction in Escher
Cljung_Model.reactions.PPA,#Add this reaction in Escher
Cljung_Model.reactions.ADK1,#Add this reaction in Escher
Cljung_Model.reactions.FRNDPR2r,#Add this reaction in Escher
#WL pathway
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2y,
Cljung_Model.reactions.ALCD2x,
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,
Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

In [12]:
for x in WLmodel.reactions:
    if x.bounds == (0, 0):{
        print(x.id)
    }

HYDFDN
HYDFDi


In [13]:
WLmodel.objective = "EX_etoh_e"
WLmodel.optimize()

,fluxes,reduced_costs
EX_h2_e,0.000000,-0.333333
EX_h2o_e,-5.000000,-0.000000
EX_co2_e,6.666667,0.000000
EX_co_e,-10.000000,-0.333333
EX_pi_e,0.000000,-0.000000
...,...,...
ACOAD1z,0.000000,-0.333333
BTCOARx,0.000000,-0.666667
ALCD4,0.000000,-0.000000
PBUTT,0.000000,0.000000


In [14]:
#WLmodel.reactions.ACt2r

In [15]:
WLmodel.objective = "EX_ac_e"
WLmodel.optimize()
#solution = WLmodel.optimize()
WLmodel.summary()
#print("Flux through objective function: ",WLmodel.objective.expression, " = ",round(WLmodel.slim_optimize(),1))
#Builder(model = WLmodel, map_json = join(data_dir,'Models/EscherMaps/WL_core.json'), reaction_data = solution.fluxes.to_dict())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# Add Acetone production pathway

In [16]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [17]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.h_c: -1.0,
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
})
print(ctfAB_ac.check_mass_balance())    #Ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + ac_c + h_c --> acac_c + accoa_c


In [18]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
    #WLmodelActn.metabolites.h_c: -1.0,
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{}
acac_c --> act_c + co2_c


In [19]:
acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    WLmodelActn.metabolites.ac_c: -1.0,
    WLmodelActn.metabolites.ac_e: 1.0
})
acT

Reaction identifier,ACt
Name,Acetate transporter
Memory address,0x07fcecc79f6d8
Stoichiometry,ac_c --> ac_e Acetate --> Acetate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [20]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

Reaction identifier,ACTt
Name,Acetone transporter
Memory address,0x07fcecce9c0b8
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [21]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07fcecceb14a8
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [22]:
WLmodelActn.add_reactions([ctfAB_ac, adc, acT, actT, act_ex])

In [23]:
WLmodelActn.objective = "EX_etoh_e"
WLmodelActn.optimize()

,fluxes,reduced_costs
EX_h2_e,0.000000,-0.333333
EX_h2o_e,-5.000000,-0.000000
EX_co2_e,6.666667,0.000000
EX_co_e,-10.000000,-0.333333
EX_pi_e,0.000000,-0.000000
...,...,...
ctfAB_ac,0.000000,0.000000
ADC,0.000000,0.000000
ACt,0.000000,-2.333333
ACTt,0.000000,0.000000


## Add synthetic Acetone pathway of LanzaTech
https://info.ornl.gov/sites/publications/Files/Pub127156.pdf

In [24]:
WLmodelActn2 = WLmodel

WLmodelActn2.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn2.add_metabolites(Metabolite(id='aap_c',formula = 'C4H5O6P', name = 'Acetoacetyl-P', compartment = 'c' ))
WLmodelActn2.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn2.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [25]:
ptb = Reaction("Ptb")
ptb.name = "Phosphate butyryl transferase"

ptb.add_metabolites({
    WLmodelActn2.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn2.metabolites.pi_c: -1.0,      #Phosphate
    WLmodelActn2.metabolites.coa_c: 1.0,      #Coenzyme A
    WLmodelActn2.metabolites.aap_c: 1.0,      #Acetoacetyl-P
})
print(ptb.check_mass_balance())    #Ensure correct mass balance
print(ptb.bounds)                  #Ensure correct bounds
print(ptb.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + pi_c --> aap_c + coa_c


In [26]:
buk = Reaction("Buk")
buk.name = "Butyrate kinase"

buk.add_metabolites({
    WLmodelActn2.metabolites.aap_c: -1.0,     #Acetoacetyl-P
    WLmodelActn2.metabolites.h_c: -1.0,       #hydrogen, not mentioned, required for balancing
    WLmodelActn2.metabolites.adp_c: -1.0,     #adp
    WLmodelActn2.metabolites.atp_c: 1.0,      #atp
    WLmodelActn2.metabolites.acac_c: 1.0,     #Acetyl-CoA
})
print(buk.check_mass_balance())    #ensure correct mass balance
print(buk.bounds)                  #Ensure correct bounds
print(buk.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aap_c + adp_c + h_c --> acac_c + atp_c


In [27]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn2.metabolites.act_c: -1.0,
    WLmodelActn2.metabolites.act_e: 1.0
})

act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn2.metabolites.act_e: -1.0,
})

WLmodelActn2.add_reactions([ptb, buk, actT, act_ex])

Ignoring reaction 'ACTt' since it already exists.
Ignoring reaction 'EX_act_e' since it already exists.


In [41]:
WLmodelActn.objective = "EX_act_e"
WLmodelActn.optimize()
#solution = WLmodel.optimize()
WLmodelActn.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co_e,EX_co_e,10,1,100.00%
h2o_e,EX_h2o_e,3.75,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
act_e,EX_act_e,-1.25,3,37.50%
co2_e,EX_co2_e,-6.25,1,62.50%


In [40]:
WLmodelActn2.objective = "EX_act_e"
WLmodelActn2.optimize()
#solution = WLmodel.optimize()
WLmodelActn2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co_e,EX_co_e,10,1,100.00%
h2o_e,EX_h2o_e,3.75,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
act_e,EX_act_e,-1.25,3,37.50%
co2_e,EX_co2_e,-6.25,1,62.50%


# Test model and visualize in Escher

In [28]:
WLmodelActn.objective = "EX_ac_e"
WLmodelActn.optimize()
#Builder(model = WLmodelActn, map_json = join(data_dir,'Models/AcetonePathway2.json'), reaction_data = solution.fluxes.to_dict())

,fluxes,reduced_costs
EX_h2_e,0.000000,-0.0
EX_h2o_e,-5.000000,-0.0
EX_co2_e,6.666667,0.0
EX_co_e,-10.000000,-0.0
EX_pi_e,0.000000,-0.0
...,...,...
ACt,0.000000,0.0
ACTt,0.000000,0.0
EX_act_e,0.000000,0.0
Ptb,0.000000,0.0


In [29]:
#Builder(model = WLmodelActn, map_json = join(data_dir,'Models/WL_core_clean.json'), reaction_data = solution.fluxes.to_dict())#
#Builder(model = , map_json = join(data_dir,'Models/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

In [30]:
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': 0.0, 'EX_h2o_e': -5.0, 'EX_ac_e': 2.5, 'EX_co2_e': 5.0, 'EX_co_e': -10.0, 'E…

In [31]:
#Builder(model = WLmodelActn)

In [32]:
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/WL_core_Actn.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': 0.0, 'EX_h2o_e': -5.0, 'EX_ac_e': 2.5, 'EX_co2_e': 5.0, 'EX_co_e': -10.0, 'E…

In [33]:
WLmodelActn.reactions

[<Reaction EX_h2_e at 0x7fcecb8df048>,
 <Reaction EX_h2o_e at 0x7fcecb8daf98>,
 <Reaction EX_co2_e at 0x7fcecba29a90>,
 <Reaction EX_co_e at 0x7fcecba30390>,
 <Reaction EX_pi_e at 0x7fcecb8f1588>,
 <Reaction EX_ac_e at 0x7fcecba054e0>,
 <Reaction EX_etoh_e at 0x7fcecba4e5c0>,
 <Reaction EX_btd_RR_e at 0x7fcecba05e10>,
 <Reaction EX_but_e at 0x7fcecba11dd8>,
 <Reaction EX_btoh_e at 0x7fcecba11400>,
 <Reaction H2td at 0x7fcecbc66a58>,
 <Reaction H2Ot at 0x7fcecbc5ffd0>,
 <Reaction CO2t at 0x7fcecba9da90>,
 <Reaction COt at 0x7fcecbaa6278>,
 <Reaction ETOHt at 0x7fcecbc12f60>,
 <Reaction BTDt_RR at 0x7fcecba79e80>,
 <Reaction BUTt at 0x7fcecba81e80>,
 <Reaction BTOHt at 0x7fcecba79be0>,
 <Reaction HYDFDN at 0x7fcecd12ec50>,
 <Reaction HYDFDi at 0x7fcecca92358>,
 <Reaction HYDFDN2r at 0x7fcecca792e8>,
 <Reaction ATPS4r at 0x7fceccec1240>,
 <Reaction ATPM at 0x7fcecba71ac8>,
 <Reaction RNF at 0x7fceccd7c438>,
 <Reaction NTPP6 at 0x7fcecd13dc88>,
 <Reaction PPA at 0x7fceccdf3f98>,
 <Reaction

In [34]:
WLmodelActn.reactions.EX_etoh_e

Reaction identifier,EX_etoh_e
Name,Ethanol exchange
Memory address,0x07fcecba4e5c0
Stoichiometry,etoh_e --> Ethanol -->
GPR,
Lower bound,0
Upper bound,1000.0


In [35]:
WLmodelActn.reactions.EX_ac_e

Reaction identifier,EX_ac_e
Name,Acetate exchange
Memory address,0x07fcecba054e0
Stoichiometry,ac_e --> Acetate -->
GPR,
Lower bound,0
Upper bound,1000.0


In [36]:
WLmodelActn.reactions.ATPM.knock_out()

In [37]:
for x in WLmodelActn.reactions:
    if x.bounds == (0, 0):{
        print(x.id)
    }

HYDFDN
HYDFDi
ATPM


In [38]:
WLmodelActn.reactions.BTOHt.bounds = -1000.0, 1000.0
WLmodelActn.reactions.HYDFDN.bounds = -1000.0, 1000.0
WLmodelActn.reactions.HYDFDi.bounds = -1000.0, 1000.0
WLmodelActn.reactions.ACACT1r.bounds = -1000.0, 1000.0
WLmodelActn.reactions.HACD1.bounds = -1000.0, 1000.0
WLmodelActn.reactions.ECOAH1.bounds = -1000.0, 1000.0
WLmodelActn.reactions.ACOAD1z.bounds = -1000.0, 1000.0
WLmodelActn.reactions.BTCOARx.bounds = -1000.0, 1000.0
WLmodelActn.reactions.ALCD4.bounds = -1000.0, 1000.0
WLmodelActn.reactions.PBUTT.bounds = -1000.0, 1000.0
WLmodelActn.reactions.BUTKr.bounds = -1000.0, 1000.0

solution = WLmodelActn.optimize()
WLmodelActn.optimize()

,fluxes,reduced_costs
EX_h2_e,0.000000,-0.0
EX_h2o_e,-5.000000,-0.0
EX_co2_e,6.666667,0.0
EX_co_e,-10.000000,-0.0
EX_pi_e,0.000000,-0.0
...,...,...
ACt,0.000000,0.0
ACTt,0.000000,0.0
EX_act_e,0.000000,0.0
Ptb,0.000000,0.0


In [39]:
WLmodelActn.objective = "EX_etoh_e"
WLmodelActn.optimize()
#WLmodelActn.summary()
#Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/WL_core_Actn.json'), reaction_data = solution.fluxes.to_dict())

,fluxes,reduced_costs
EX_h2_e,0.000000,-0.333333
EX_h2o_e,-5.000000,-0.000000
EX_co2_e,6.666667,0.000000
EX_co_e,-10.000000,-0.333333
EX_pi_e,0.000000,-0.000000
...,...,...
ACt,0.000000,0.000000
ACTt,0.000000,0.000000
EX_act_e,0.000000,0.000000
Ptb,0.000000,-2.666667


# Alternative: build from Moorella model

Builder(model = Moorella_Model)#, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'))

Moorella_Model.metabolites.get_by_id("10fthf_c")

Moorella_Model.metabolites.methf_c

Moorella_Model.metabolites.mlthf_c

Moorella_Model.reactions.TMDS

CO2t
CODH_ACS
PFOR
Moorella_Model.reactions.CODH

#Transport & Exchange:
Moorella_Model.reactions.EX_co_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_co2_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_h2_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_ac_LPAREN_e_RPAREN
Moorella_Model.reactions.COt
Moorella_Model.reactions.CO2t
Moorella_Model.reactions.H2td
Moorella_Model.reactions.ACt2r

#Energy:
Moorella_Model.reactions.HYDFDN
Moorella_Model.reactions.CODH

#Eastern branch:
Moorella_Model.reactions.FDH

#Western branch:
Moorella_Model.reactions.CODH_ACS


WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([Moorella_Model.reactions.CODH])

WLmodel = Model('Wood_Ljungdahl_Core')

reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default